In [24]:
import boto3
import pandas as pd
import os
from dotenv import load_dotenv
from io import StringIO

In [ ]:
load_dotenv()
s3 = boto3.client(
    's3',
    aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
    aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'),
    region_name=os.getenv("AWS_DEFAULT_REGION")
)

bucket = os.getenv('S3_BUCKET')
input_key = 'raw_sales_data.csv'
output_key = 'cassey_processed/processed_electronics_sales.csv'

# Extract

In [26]:
obj = s3.get_object(Bucket=bucket, Key=input_key)
raw_data = pd.read_csv(StringIO(obj['Body'].read().decode('utf-8')))
print(raw_data.head())

   sale_id  product_id             product_name     category  quantity_sold  \
0        1         101               Laptop Pro  Electronics              5   
1        2         102              Desktop Max  Electronics              2   
2        3         201             Office Chair    Furniture             10   
3        4         103           Wireless Mouse  Electronics             25   
4        5         301  Python Programming Book        Books             50   

   price_per_unit   sale_date region  
0          1200.0  2023-01-15  North  
1          1500.0  2023-01-16  North  
2           150.0  2023-01-17  South  
3            25.0  2023-01-18   West  
4            45.0  2023-01-18   East  


# Transform

In [27]:
# Calculate a new column total_revenue (quantity_sold * price_per_unit).
raw_data['total_revenue'] = raw_data['quantity_sold'] * raw_data['price_per_unit']
# print(raw_data.head())

In [28]:
# Filter data to include only sales from the 'Electronics' category.
filtered_data = raw_data[raw_data['category'] == 'Electronics']
# print(filtered_data.head())

In [32]:
# Ensure sale_date is in YYYY-MM-DD format
filtered_data.loc[:, 'sale_date'] = pd.to_datetime(filtered_data['sale_date']).dt.strftime('%Y-%m-%d')


In [30]:
# Select and reorder columns: sale_id, product_name, category, total_revenue, sale_date, region
filtered_data = filtered_data[['sale_id', 'product_name', 'category', 'total_revenue', 'sale_date', 'region']]
print(filtered_data.head())

   sale_id    product_name     category  total_revenue   sale_date region
0        1      Laptop Pro  Electronics         6000.0  2023-01-15  North
1        2     Desktop Max  Electronics         3000.0  2023-01-16  North
3        4  Wireless Mouse  Electronics          625.0  2023-01-18   West
5        6      Laptop Pro  Electronics         3600.0  2023-01-19  South
7        8    Keyboard Pro  Electronics         1125.0  2023-01-20   East


# Load

In [33]:
# Save CSV back to S3
csv_buffer = StringIO()
filtered_data.to_csv(csv_buffer, index=False)

s3.put_object(Bucket=bucket, Key=output_key, Body=csv_buffer.getvalue())
print("Processed file uploaded to S3.")

Processed file uploaded to S3.
